# Map the sweep IDs to names using the wanbd API

In [2]:
import wandb
import pandas as pd

# === Settings ===
ENTITY = "matteotolloso"        # e.g. "my-team" or your username
PROJECT = "graph-uncertainty"      # e.g. "my-cool-project"
INPUT_CSV = "/Users/matteotolloso/graph-uncertainty/wandb_export_2025-11-18T20_57_43.211+01_00.csv"
OUTPUT_CSV = "./wandb_export_final.csv"

# === Load CSV ===
df = pd.read_csv(INPUT_CSV)

# Get unique sweep IDs from the CSV
sweep_ids = df["Sweep"].dropna().unique()

# === Fetch sweep names from API ===
api = wandb.Api()
id_to_name = {}

for sid in sweep_ids:
    try:
        sweep = api.sweep(f"{ENTITY}/{PROJECT}/{sid}")
        id_to_name[sid] = sweep.name or sid  # fallback to ID if no name
    except Exception as e:
        print(f"⚠️ Could not fetch sweep {sid}: {e}")
        id_to_name[sid] = sid  # fallback to ID

# === Replace IDs with names ===
df["Sweep"] = df["Sweep"].map(id_to_name)

# === Save new CSV ===
df.to_csv(OUTPUT_CSV, index=False)

print(f"✅ Updated CSV saved as {OUTPUT_CSV}")


✅ Updated CSV saved as ./wandb_export_final.csv


# Differences between different conv layers

In [13]:
import pandas as pd

df = pd.read_csv("./wandb_export_final.csv")

In [14]:
# take the df where the Sweep is something ending with "credal_LJ"
df = df[df["Sweep"].str.endswith("credal_LJ", na=False)]
# add the column "dataset" that is the part of the Sweep before the first underscore
# avoid SettingWithCopyWarning by working on a copy
df = df.copy()
df["dataset"] = df["Sweep"].str.split("_").str[0]

In [19]:
# take top 3 rows by 'test_auroc_EU' for each (dataset, gnn_type), then compute means
best_results = (
	df.sort_values(['dataset', 'gnn_type', 'test_auroc_EU'], ascending=[True, True, False])
	  .groupby(['dataset', 'gnn_type'], as_index=False)
	  .head(3)
	  .reset_index(drop=True)
)

# mean across all datasets per gnn_type (using the selected top-3 per dataset)
mean_results = best_results.groupby('gnn_type', as_index=False)['test_auroc_EU'].mean()

# (optional) mean per dataset and gnn_type (i.e. mean of the top-3 within each dataset)
mean_per_dataset = best_results.groupby(['dataset', 'gnn_type'], as_index=False)['test_auroc_EU'].mean()

In [20]:
mean_per_dataset

,dataset,gnn_type,test_auroc_EU
0,amazon,GCN,0.493302
1,amazon,SAGE,0.514767
2,arxiv,GCN,0.545215
3,arxiv,SAGE,0.627238
4,coauthor,GCN,0.678341
5,coauthor,SAGE,0.678044
6,cora,GCN,0.591474
7,cora,SAGE,0.582256
8,patents,GCN,0.587939
9,patents,SAGE,0.545788


In [23]:
# compute the percentage difference for the two different gnn_types per dataset
mean_per_dataset_pivot = mean_per_dataset.pivot(index='dataset', columns='gnn_type', values='test_auroc_EU').reset_index()
mean_per_dataset_pivot['percentage_diff'] = (
	(mean_per_dataset_pivot['GCN'] - mean_per_dataset_pivot['SAGE']) / mean_per_dataset_pivot['SAGE']
) * 100

In [24]:
mean_per_dataset_pivot

gnn_type,dataset,GCN,SAGE,percentage_diff
0,amazon,0.493302,0.514767,-4.169915
1,arxiv,0.545215,0.627238,-13.076767
2,coauthor,0.678341,0.678044,0.043912
3,cora,0.591474,0.582256,1.583090
4,patents,0.587939,0.545788,7.722954
5,reddit2,0.588319,0.668222,-11.957522
6,roman,0.597036,0.592306,0.798639
7,squirrel,0.474251,0.641163,-26.032726


# differences betweeen num layers

In [31]:
# similar study as above, but with the number of layers instead of gnn_type
import pandas as pd

df = pd.read_csv("./wandb_export_final.csv")

# take the df where the Sweep is something ending with "credal"
df = df[df["Sweep"].str.endswith("credal", na=False)]
# add the column "dataset" that is the part of the Sweep before the first underscore
# avoid SettingWithCopyWarning by working on a copy
df = df.copy()
df["dataset"] = df["Sweep"].str.split("_").str[0]

# filter num_layers to be only 2 or 3
df = df[df["num_layers"].isin([2, 3])]

In [32]:
# take top 3 rows by 'test_auroc_EU' for each (dataset, num_layers), then compute means
best_results = (
	df.sort_values(['dataset', 'num_layers', 'test_auroc_EU'], ascending=[True, True, False])
	  .groupby(['dataset', 'num_layers'], as_index=False)
	  .head(3)
	  .reset_index(drop=True)
)

# mean across all datasets per gnn_type (using the selected top-3 per dataset)
mean_results = best_results.groupby('gnn_type', as_index=False)['test_auroc_EU'].mean()

# (optional) mean per dataset and gnn_type (i.e. mean of the top-3 within each dataset)
mean_per_dataset = best_results.groupby(['dataset', 'num_layers'], as_index=False)['test_auroc_EU'].mean()

In [33]:
mean_per_dataset

,dataset,num_layers,test_auroc_EU
0,amazon,2.0,0.492513
1,amazon,3.0,0.507011
2,arxiv,2.0,0.546279
3,arxiv,3.0,0.552851
4,coauthor,2.0,0.511099
5,coauthor,3.0,0.503521
6,cora,3.0,0.583782
7,patents,2.0,0.551532
8,patents,3.0,0.640777
9,reddit2,2.0,0.637856


In [39]:
# compute the percentage difference for the two different gnn_types per dataset
mean_per_dataset_pivot = mean_per_dataset.pivot(index='dataset', columns='num_layers', values='test_auroc_EU').reset_index()
mean_per_dataset_pivot['percentage_diff'] = (
	(mean_per_dataset_pivot[2] - mean_per_dataset_pivot[3]) / mean_per_dataset_pivot[3]
) * 100
mean_per_dataset_pivot

num_layers,dataset,2.0,3.0,percentage_diff
0,amazon,0.492513,0.507011,-2.859564
1,arxiv,0.546279,0.552851,-1.188656
2,coauthor,0.511099,0.503521,1.505014
3,cora,NaN,0.583782,NaN
4,patents,0.551532,0.640777,-13.927703
5,reddit2,0.637856,0.551317,15.696780
6,roman,0.660266,0.529944,24.591645
